In [1]:
import numpy as np
import random
import os
from PIL import Image
from numpy.linalg import norm
from sklearn.utils import shuffle

In [2]:
class Activation:
    def relu(self, X):
        """
        Memberikan nilai output berupa fungsi ReLU dari X
        
        Parameter:
        X: Numpy 2D Array berukuran (jumlah fitur, jumlah data)
        
        Return:
        Y: Numpy 2D Array yang merupakan output dari fungsi ReLU
        """
        Y = np.maximum(X, 0)
        return Y
    
    def softmax(self, X):
        """
        Memberikan nilai output berupa fungsi Softmax dari X
        
        Parameter:
        X: Numpy 2D Array berukuran (jumlah fitur, jumlah data)
        
        Return:
        Y: Numpy 2D Array yang merupakan output dari fungsi Softmax
        """
        # Menghindari overflow (https://www.techopedia.com/definition/663/overflow-error#:~:text=In%20computing%2C%20an%20overflow%20error,of%20its%20ability%20to%20handle.)
        X -= np.max(X, axis=0, keepdims=True)
        # Perhitungan
        exp_X = np.exp(X)
        sum_exp_X = np.sum(exp_X, axis=0, keepdims=True)
        Y = exp_X / sum_exp_X
        return Y

In [3]:
class Metrics:
    def categorical_crossentropy(self, Y_pred, Y_true):
        """
        Memberikan nilai output metrik categorical crossentropy dengan membandingkan Y_pred dengan Y_true
        
        Parameter:
        Y_pred: Numpy 2D Array berukuran (jumlah fitur, jumlah data) yang merupakan hasil prediksi
        Y_true: Numpy 2D Array berukuran (jumlah fitur, jumlah data) yang merupakan nilai target pada data
        
        Return:
        cost: Nilai skalar yang merupakan hasil output dari categorical crossentropy
        """
        # Perhitungan
        cross_entropy = -np.sum(Y_true * np.log(Y_pred))
        cost = cross_entropy / Y_true.shape[1]
        return cost

    def accuracy(self, Y_pred, Y_true):
        """
        Memberikan nilai output metrik akurasi
        
        Parameter:
        Y_pred: Numpy 2D Array berukuran (jumlah fitur, jumlah data) yang merupakan hasil prediksi
        Y_true: Numpy 2D Array berukuran (jumlah fitur, jumlah data) yang merupakan nilai target pada data
        
        Return:
        acc: Nilai skalar yang merupakan hasil output dari akurasi
        """
        pred_matrix = Y_pred == np.max(Y_pred, axis=0)
        acc = np.sum(pred_matrix * Y_true) / Y_pred.shape[1]
        return acc

In [4]:
def data_preprocessing(DIR, image_size=(100, 100)):
    images_matrix = []
    labels_matrix = []
    for subdir in os.listdir(DIR):
        SUBDIR = DIR + f"/{subdir}"
        for image_file in os.listdir(SUBDIR):
            # Labels
            label_vector = np.zeros((len(TRUE_LABELS)))
            label_idx = TRUE_LABELS[subdir]
            label_vector[label_idx] = 1
            labels_matrix.append(label_vector)
            # Images
            image = Image.open(f"{SUBDIR}/{image_file}")
            image = image.resize(image_size)
            image = np.asarray(image, dtype=np.float64)
            image /= 255.0
            images_matrix.append(image)
    images_matrix = np.expand_dims(images_matrix, axis=-1)
    labels_matrix = np.array(labels_matrix)
    
    # Randomize the data
    random_num = random.randint(0, 42)
    X, Y = shuffle(images_matrix, labels_matrix, random_state=random_num)
    return X, Y.T

In [5]:
def convolution(X, W, B=np.array(0)):
    conv_images = np.zeros((X.shape[0], X.shape[1] - W.shape[1] + 1, \
                                 X.shape[2] - W.shape[2] + 1, W.shape[0]))
    img_len, conv_x, conv_y, filters = conv_images.shape
    for idx in range(img_len):
        for i in range(conv_x):
            for j in range(conv_y):
                sub_image = X[idx, i:i+W.shape[1], j:j+W.shape[2], :]
                conv_images[idx, i, j, :] = np.sum(sub_image * W, axis=(1, 2, 3)) + B.flatten()
    return conv_images

In [6]:
def pooling(X, pool_size=(2, 2)):
    """
    Mengembalikan matriks citra hasil max pooling

    Parameter:
    X: Matriks Input Numpy Array 4D berukuran (jumlah data, panjang pixel, lebar pixel, anchor)
    pool_size: Ukuran sub-matriks Numpy Array 2D yang akan diterapkan operasi max pooling

    Return:
    pooled_images: Matriks Pooling Numpy Array 4D
    """
    img_len, pooled_x, pooled_y, anchors = X.shape[0], X.shape[1] // pool_size[0], X.shape[2] // pool_size[1], X.shape[-1]
    pooled_images = np.zeros((img_len, pooled_x, pooled_y, anchors))

    for idx in range(img_len):  
        for i in range(0, pooled_x*pool_size[0], pool_size[0]):
            for j in range(0, pooled_y*pool_size[1], pool_size[1]):
                sub_image = X[idx, i:i+pool_size[0], j:j+pool_size[1], :]
                pooled_images[idx, int(i/pool_size[0]), int(j/pool_size[1]), :] = np.max(sub_image, axis=(0, 1))
    return pooled_images

In [11]:
TRAIN_DIR = './New Data - Copy/Train'
TEST_DIR = './New Data - Copy/Test'
dirs = (TRAIN_DIR, TEST_DIR)

TRUE_LABELS = {}

# Assign each photo to corresponding labels
for directory in dirs:
    for label, subdir in enumerate(os.listdir(directory)):
        TRUE_LABELS[subdir] = label
        
train_images, train_labels = data_preprocessing(TRAIN_DIR, image_size=(14, 14))

In [12]:
train_images = train_images[-10:]
train_labels = train_labels[:, -10:]

In [13]:
W1 = np.random.randn(4, 3, 3, 1) * 0.01
B1 = np.random.randn(4, 1) * 0.01

W2 = np.random.randn(16, 3, 3, 4) * 0.01
B2 = np.random.randn(16, 1) * 0.01

W3 = np.random.randn(64, 64) * 0.01
B3 = np.random.randn(64, 1) * 0.01

W4 = np.random.randn(3, 64) * 0.01
B4 = np.random.randn(3, 1) * 0.01

## Fitting

In [92]:
parameter = B1.copy()
dB1_approx = np.zeros_like(parameter)

for i in range(parameter.shape[0]):
    for j in range(parameter.shape[1]):
#         for k in range(parameter.shape[2]):
#             for l in range(parameter.shape[3]):
        E = np.zeros_like(parameter)
        E[i, j] = 1

        # Convolutional Layer
        z1 = convolution(train_images, W1, B1)
        a1 = Activation().relu(z1)

        z1_plus = convolution(train_images, W1, B1 + 1e-7 * E)
        a1_plus = Activation().relu(z1_plus)

        z1_minus = convolution(train_images, W1, B1 - 1e-7 * E)
        a1_minus = Activation().relu(z1_minus)

        # Max Pooling Layer
        a2 = pooling(a1)
        a2_plus = pooling(a1_plus)
        a2_minus = pooling(a1_minus)

        # Convolution Layer
        z3 = convolution(a2, W2, B2)
        a3 = Activation().relu(z3)

        z3_plus = convolution(a2_plus, W2, B2)
        a3_plus = Activation().relu(z3_plus)

        z3_minus = convolution(a2_minus, W2, B2)
        a3_minus = Activation().relu(z3_minus)

        # Max Pooling Layer
        a4 = pooling(a3)
        a4_plus = pooling(a3_plus)
        a4_minus = pooling(a3_minus)

        # Flatten
        a5 = np.reshape(a4, (a4.shape[0], -1)).T
        a5_plus = np.reshape(a4_plus, (a4.shape[0], -1)).T
        a5_minus = np.reshape(a4_minus, (a4.shape[0], -1)).T

        # Dense 1
        z6 = W3 @ a5 + B3
        a6 = Activation().relu(z6)

        z6_plus = (W3) @ a5_plus + (B3)
        a6_plus = Activation().relu(z6_plus)

        z6_minus = (W3) @ a5_minus + (B3)
        a6_minus = Activation().relu(z6_minus)

        # Dense 2
        z7 = W4 @ a6 + B4
        a7 = Activation().softmax(z7)

        z7_plus = (W4) @ a6_plus + (B4)
        a7_plus = Activation().softmax(z7_plus)

        z7_minus = (W4) @ a6_minus + (B4)
        a7_minus = Activation().softmax(z7_minus)

        # Gradient Checking
        loss_plus = Metrics().categorical_crossentropy(a7_plus, train_labels)
        loss_minus = Metrics().categorical_crossentropy(a7_minus, train_labels)

        dB1_approx[i, j] = (loss_plus - loss_minus) / 2e-7
    print(f"Done {i+1}")

Done 1
Done 2
Done 3
Done 4


### Backpropagation

In [79]:
img = a7.shape[1]
dz7 = a7 - train_labels
dW4 = dz7 @ a6.T / img 
dB4 = np.sum(dz7, axis=1, keepdims=True) / img

In [80]:
diff_z6 = np.where(z6 >= 0, 1, 0)
dz6 = (W4.T @ dz7) * diff_z6
dW3 = dz6 @ a5.T / img
dB3 = np.sum(dz6, axis=1, keepdims=True) / img

In [81]:
dz5 = W3.T @ dz6

In [82]:
dz4 = np.reshape(dz5.T, newshape=a4.shape)

In [83]:
dC = np.zeros_like(a3)
img_len, m, n, anchors = a3.shape

# Jika ukuran output adalah ganjil
if m % 2 == 1:
    m -= 1
if n %2 == 1:
    n -= 1

for idx in range(img_len):
    for i in range(0, m, 2):
        for j in range(0, n, 2):
            sub_C = a3[idx, i:i+2, j:j+2, :]
            dC[idx, i:i+2, j:j+2, :] = np.where(sub_C == np.max(sub_C, axis=(0, 1), keepdims=True),
                                                np.reshape(dz4[idx, int(i/2), int(j/2), :], (1, 1, -1)), 0) /\
                                       np.where(np.sum(sub_C == np.max(sub_C, axis=(0, 1), keepdims=True), axis=(0, 1), keepdims=True) > 1, 2, 1)

# dC/dZ
dCdZ = np.where(z3 >= 0, 1, 0)

# dZ = dcost/dZ
dz3 = dC * dCdZ

dW2 = convolution(a2.T, dz3.T).T / img_len
dB2 = np.expand_dims(np.sum(dz3, axis=(0, 1, 2)), axis=-1) / img_len

In [84]:
padded_size = 2
padded_dZ = np.pad(dz3, ((0, 0), (padded_size, padded_size), (padded_size, padded_size), (0, 0)))
rotated_W = np.rot90(W2, axes=(1, 2), k=2)
dz2 = convolution(X=padded_dZ, W=rotated_W.swapaxes(0, -1))

In [85]:
dC = np.zeros_like(a1)
img_len, m, n, anchors = a1.shape

# Jika ukuran output adalah ganjil
if m % 2 == 1:
    m -= 1
if n %2 == 1:
    n -= 1

for idx in range(img_len):
    for i in range(0, m, 2):
        for j in range(0, n, 2):
            sub_C = a1[idx, i:i+2, j:j+2, :]
            dC[idx, i:i+2, j:j+2, :] = np.where(sub_C == np.max(sub_C, axis=(0, 1), keepdims=True),
                                                np.reshape(dz2[idx, int(i/2), int(j/2), :], (1, 1, -1)), 0) /\
                                       np.where(np.sum(sub_C == np.max(sub_C, axis=(0, 1), keepdims=True), axis=(0, 1), keepdims=True) > 1, 2, 1)
            
# dC/dZ
dCdZ = np.where(z1 >= 0, 1, 0)

# dZ = dcost/dZ
dz1 = dC * dCdZ

dW1 = convolution(train_images.T, dz1.T).T / img
dB1 = np.expand_dims(np.sum(dz1, axis=(0, 1, 2)), axis=-1) / img

### Hasil Gradient Checking

#### Dense 2

In [18]:
norm(dz7_approx - dz7 / img) / (norm(dz7_approx) + norm(dz7 / img))

6.127957484298917e-09

In [21]:
norm(dW4_approx - dW4) / (norm(dW4_approx) + norm(dW4))

3.4831166319301284e-07

In [24]:
norm(dB4_approx - dB4) / (norm(dB4_approx) + norm(dB4))

7.758099447419225e-09

#### Dense 1

In [28]:
norm(dz6_approx - dz6 / img) / (norm(dz6_approx) + norm(dz6 / img))

3.7148207080823337e-07

In [30]:
norm(dW3_approx - dW3) / (norm(dW3_approx) + norm(dW3))

2.7294503335195843e-05

In [32]:
norm(dB3_approx - dB3) / (norm(dB3_approx) + norm(dB4))

2.5705167304957233e-08

#### Flatten

In [36]:
norm(dz5_approx - dz5 / img) / (norm(dz5_approx) + norm(dz5 / img))

7.830466768379444e-06

#### Max Pooling 2

In [40]:
norm(dz4_approx - dz4 / img) / (norm(dz4_approx) + norm(dz4 / img))

7.830466768379444e-06

#### Convolutional Layer 2

In [43]:
norm(da3_approx - dC / img) / (norm(da3_approx) + norm(dC / img))

1.5486409591515676e-05

In [70]:
norm(dz3_approx - dz3 / img) / (norm(dz3_approx) + norm(dz3 / img))

4.335795947955409e-06

In [52]:
norm(dW2_approx - dW2) / (norm(dW2_approx) + norm(dW2))

9.393350100422645e-05

In [58]:
norm(dB2_approx - dB2) / (norm(dB2_approx) + norm(dB2))

5.475530188681731e-06

#### Max Pooling Layer 1

In [63]:
norm(dz2_approx - dz2 / img) / (norm(dz2_approx) + norm(dz2 / img))

0.00036863762060196457

#### Convolutional Layer 1

In [75]:
norm(da1_approx - dC / img) / (norm(da1_approx) + norm(dC / img))

0.0006994379695956223

In [86]:
norm(dz1_approx - dz1 / img) / (norm(dz1_approx) + norm(dz1 / img))

0.0005277462897791574

In [90]:
norm(dW1_approx - dW1) / (norm(dW1_approx) + norm(dW1))

6.0730961500408985e-05

In [93]:
norm(dB1_approx - dB1) / (norm(dB1_approx) + norm(dB1))

0.5852971516203433